Notebook for analysing ship traffic and routes data for Liverpool

In [7]:
import sys
import os
# Add the parent directory of 'src' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

In [8]:
from src.database.mongo_connection import db
# Test MongoDB connection and list collections
collections = db.list_collection_names()
print("Collections in DB:", collections)

Collections in DB: ['vessel_position', 'latest_positions', 'ports', 'sessions', 'vessel_collections_metadata', 'vessel_images', 'vessel_details']


In [19]:
import pandas as pd

# Aggregate unique Vessel_Name and associated MMSI from vessel_details collection
pipeline = [
    {
        "$group": {
            "_id": "$Name",
            "mmsi_list": {"$addToSet": "$mmsi"},
            "destination_list": {"$addToSet": "$Destination"}
        }
    }
]
results = list(db.vessel_details.aggregate(pipeline))

# Convert to DataFrame
df_vessel_mmsi = pd.DataFrame({
    "Vessel_Name": [r["_id"] for r in results],
    "MMSI_List": [r["mmsi_list"] for r in results],
    "Destination_List": [r["destination_list"] for r in results]
})
df_vessel_mmsi.set_index("Vessel_Name", inplace=True)
df_vessel_mmsi.head(50)  # Display the first 10 rows

,MMSI_List,Destination_List
Vessel_Name,,
POLARIS,[305004000],[GDYNIA VIA KIEL]
VS PRIDE,[232012089],[CAMTR]
MH DAISEN,[563254300],[NL AMS]
ATLANTIC SUNFLOWER,[538011177],[CHARLESTON-USA]
CAPE SCOTT,[255916278],[FRLEH]
PACIFIC EGRET,[235076846],[GB BIF]
F/V MARIBELISE,[228091900],[CHERBOURG]
ARKLOW RAVEN,[250001109],[FRBAY]
SOLAR SKYLER,[538008706],[GBIMM]


In [20]:
df_vessel_mmsi.to_csv("vessel_mmsi_list_with destination.csv")

In [23]:
from src.database.mongo_connection import db
import pandas as pd

# Define Liverpool-related destination patterns
regex_patterns = [
    r"^GBLIV$",
    r"^LIVERPOOL$",
    r"^Liverpool$",
    r"^GBLIV.*",
    r".*GBLIV$",
    r".*GBLIV.*",
    r".*LIV$",
    r"^LIV.*",
    r".*LIVERPOOL.*"
]

# Build MongoDB $or regex query
query = {
    "$or": [
        { "Destination": { "$regex": pattern, "$options": "i" } }
        for pattern in regex_patterns
    ]
}

# Query vessel_details
results = list(db.vessel_details.find(query))

# Build DataFrame rows
records = []
for doc in results:
    dim = doc.get("Dimension", {})
    length = (dim.get("ToBow") or 0) + (dim.get("ToStern") or 0)
    width = (dim.get("ToPort") or 0) + (dim.get("ToStarboard") or 0)
    
    records.append({
        "mmsi": doc.get("mmsi"),
        "name": doc.get("Name"),
        "destination": doc.get("Destination"),
        "imo": doc.get("ImoNumber"),
        "call_sign": doc.get("Callsign"),
        "ship_type": doc.get("Type"),
        "draught": doc.get("draught"),
        "length_m": length,
        "width_m": width,
        "last_updated": doc.get("last_updated")
    })

# Convert to DataFrame
df = pd.DataFrame(records)

# Save to CSV
df.to_csv("liverpool_vessels.csv", index=False)

print(f"Saved {len(df)} vessels to liverpool_vessels.csv")


Saved 56 vessels to liverpool_vessels.csv
